In [1]:
import os
import multiprocessing
import numpy as np
import cv2
import time
import shutil

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import transforms

In [2]:
root_dir = '/content/drive/MyDrive/Models_exp2.14.3'

if torch.cuda.is_available():  
  dev = "cuda:0" 
else:  
  dev = "cpu"  

device = torch.device(dev)
print(device)

#https://machinelearningmastery.com/pytorch-tutorial-develop-deep-learning-models/
class Model_Encoder(nn.Module):
    '''
    A model to use on 3d objects
    '''
    def __init__(self):
      super(Model_Encoder, self).__init__()
      self.inception = torchvision.models.inception_v3(pretrained=False, aux_logits=False) #https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html
      
    def forward(self, x):
      
      x =  nn.functional.interpolate(x, 299)

      x = self.inception(x)


      return x



cuda:0


In [3]:
class Date_3d_Cached(torch.utils.data.Dataset):
    def __init__(self, x, y):
      self.x = torch.from_numpy(x)
      self.y = torch.from_numpy(y)
    
    def __len__(self):
      return self.x.shape[0]

    def __getitem__(self, idx):
      x = self.x[idx] /255.0
      return x, self.y[idx]

views = ['bottom', 'side_1', 'side_2', 'side_3', 'side_4', 'top', 'top_1', 'top_2', 'top_3', 'top_4', 'bot_1', 'bot_2', 'bot_3', 'bot_4']
x = np.load('/content/drive/MyDrive/Cache/x_compressed.npz')['arr_0']
y = np.load('/content/drive/MyDrive/Cache/y.npy')

training_data = Date_3d_Cached(x, y)

In [4]:
encoder = torch.load('/content/drive/MyDrive/Models_exp2.14.2/ep_80_loss_90.1415479183197val_loss_best.pt')

In [5]:
encoder.eval()

for param in encoder.parameters():
    param.requires_grad = False

In [6]:
#https://machinelearningmastery.com/pytorch-tutorial-develop-deep-learning-models/
class Model_3d(nn.Module):
    '''
    A model to use on 3d objects
    '''
    def __init__(self, fc_nodes, no_views, no_classes, encoder):
      super(Model_3d, self).__init__()
      self.encoder = encoder

      num_features = 1000

      self.fcs = nn.ModuleList()

      for idx in range(no_views):
        self.fcs.append(nn.Linear(num_features, fc_nodes))

      first_dense_count = fc_nodes * no_views

      self.dense_1 = nn.Linear(first_dense_count, no_classes)

    
    def forward(self, x):
      features = []

      temp = x.view([-1, 3, 224, 224])
      
      temp =  nn.functional.interpolate(temp, 299)

      x = temp.view([x.shape[0], x.shape[1], 3, 299, 299])

      for i, fc in enumerate(self.fcs):
        sample = x[:, i, ...]
        r_out = self.encoder(sample)
        fc_out = fc(r_out)
        features.append(fc_out)
      
      stack = torch.stack(features, 1)

      reshaped = stack.view([x.shape[0], -1])

      x = self.dense_1(reshaped)


      return torch.nn.functional.softmax(x)

In [7]:
model = Model_3d(1024, len(views), 10, encoder)
model.to(device)
print(device)

cuda:0


In [8]:
encoder.to(device)

Model_Encoder(
  (inception): Inception3(
    (Conv2d_1a_3x3): BasicConv2d(
      (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
      (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    )
    (Conv2d_2a_3x3): BasicConv2d(
      (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    )
    (Conv2d_2b_3x3): BasicConv2d(
      (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    )
    (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (Conv2d_3b_1x1): BasicConv2d(
      (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    )
    (Conv2d_4a_3x3):

In [9]:
#90/10 train val split
train_len = int(len(training_data) * .9)
val_len = len(training_data) - train_len

batch_size = 8

In [10]:
train_set, val_set = torch.utils.data.random_split(training_data, [train_len, val_len],torch.Generator().manual_seed(42))
training_loader = torch.utils.data.DataLoader(train_set, batch_size= batch_size, shuffle=True, num_workers=os.cpu_count())
val_loader = torch.utils.data.DataLoader(val_set, batch_size= batch_size, shuffle=True, num_workers=os.cpu_count())

#https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [11]:
#https://medium.com/analytics-vidhya/saving-and-loading-your-model-to-resume-training-in-pytorch-cb687352fa61
def save_ckp(state, model_path):
    torch.save(state, model_path)

In [ ]:
if not os.path.exists(root_dir):
    os.makedirs(root_dir)


low_val_loss = 30000000
best_acc = -1
epoch = 0

val_loss_path = None
acc_path = None
#https://stackoverflow.com/questions/8078330/csv-writing-within-loop
import csv
with open(os.path.join(root_dir, 'logs.csv'), 'w') as file_csv:
  writer=csv.writer(file_csv, delimiter=',',lineterminator='\n',)
  writer.writerow(['epoch', 'loss', 'acc', 'val_loss', 'val_acc'])


  for epoch in range(1, 101):
      row = [epoch]
      is_best = False
      running_loss = 0.0
      correct = 0
      total = 0

      model.train()

      t0 = time.time()
      for i, data in enumerate(training_loader):
          # get the inputs; data is a list of [inputs, labels]
          inputs, labels = data

          # for this_view in range(inputs.shape[1]):
          #   if np.random.randint(0, 4) == 0:
          #     inputs[:, this_view, ...] = 1.0

          inputs = inputs.to(device)
          labels = labels.to(device)

          # zero the parameter gradients
          optimizer.zero_grad()

          # forward + backward + optimize
          outputs = model(inputs)
          loss = criterion(outputs, labels)
          loss.backward()
          optimizer.step()

          # print statistics
          correct += (outputs.argmax(1) == labels).float().sum() #https://stackoverflow.com/questions/51503851/calculate-the-accuracy-every-epoch-in-pytorch
          total += labels.shape[0]
          running_loss += loss.item()

      print('epoch', epoch)
      
      print('{} seconds'.format(time.time() - t0))
      print('loss', running_loss)
      row.append(running_loss)
      running_loss = 0.0

      accuracy = 100 * correct / total
      print("Accuracy = {}".format(accuracy))
      row.append(torch.IntTensor.item(accuracy))
      correct = 0
      total= 0

      running_loss = 0.0
      correct = 0
      total = 0

      optimizer.zero_grad()
      model.eval()
      t0 = time.time()
      with torch.no_grad():
          for i, data in enumerate(val_loader):
              
              # get the inputs; data is a list of [inputs, labels]
              inputs, labels = data
              inputs = inputs.to(device)
              labels = labels.to(device)        


              # forward + backward + optimize
              outputs = model(inputs)
              loss = criterion(outputs, labels)

              correct += (outputs.argmax(1) == labels).float().sum() #https://stackoverflow.com/questions/51503851/calculate-the-accuracy-every-epoch-in-pytorch
              total += labels.shape[0]
              running_loss += loss.item()

      print('{} seconds'.format(time.time() - t0))
      print('val loss', running_loss)
      row.append(running_loss)
      

      accuracy = 100 * correct / total
      print(" Val Accuracy = {}".format(accuracy))
      row.append(torch.IntTensor.item(accuracy))

      model_name = 'ep_' + str(epoch) + '_loss_' + str(running_loss) + '_acc_' + str(accuracy) + '.pt'
      full_model_path = os.path.join(root_dir, model_name)
      checkpoint = {
          'epoch': epoch + 1,
          'state_dict': model.state_dict(),
          'optimizer': optimizer.state_dict()
      }
      if running_loss < low_val_loss:
        if val_loss_path and os.path.exists(val_loss_path):
          open(val_loss_path, 'w').close() #overwrite and make the file blank instead - ref: https://stackoverflow.com/a/4914288/3553367
          os.remove(val_loss_path)
        val_loss_path = full_model_path.replace('.pt', 'val_loss_best.pt')
        save_ckp(model, val_loss_path)
        low_val_loss = running_loss
        print('new low loss')

      if accuracy > best_acc:
        if acc_path and os.path.exists(acc_path):
          open(acc_path, 'w').close() #overwrite and make the file blank instead - ref: https://stackoverflow.com/a/4914288/3553367
          os.remove(acc_path)
        acc_path = full_model_path.replace('.pt', 'acc_best.pt')
        save_ckp(model, acc_path)
        best_acc = accuracy
        print('new best acc')
      full_model_path = os.path.join(root_dir, 'last.pt')
      save_ckp(checkpoint, full_model_path)
          

      

      correct = 0
      total= 0
      running_loss = 0.0
      writer.writerow(row)
  print('Finished Training')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:44: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


epoch 1
209.04812479019165 seconds
loss 952.1037695407867
Accuracy = 33.50041961669922
21.756895542144775 seconds
val loss 109.16125786304474
 Val Accuracy = 27.75
new low loss
new best acc
epoch 2
210.88076829910278 seconds
loss 943.9668000936508
Accuracy = 35.561126708984375
21.75898814201355 seconds
val loss 103.36816990375519
 Val Accuracy = 38.75
new low loss
new best acc
epoch 3
211.65215063095093 seconds
loss 940.5476417541504
Accuracy = 36.45224380493164
21.814759016036987 seconds
val loss 102.988285779953
 Val Accuracy = 40.0
new low loss
new best acc
epoch 4
211.86609554290771 seconds
loss 939.5795533657074
Accuracy = 36.59148025512695
21.87909698486328 seconds
val loss 104.91531753540039
 Val Accuracy = 35.75
epoch 5
210.34031128883362 seconds
loss 937.8158284425735
Accuracy = 37.06488800048828
21.832563638687134 seconds
val loss 103.10736310482025
 Val Accuracy = 39.75
epoch 6
210.0778784751892 seconds
loss 937.6181879043579
Accuracy = 37.009193420410156
21.82516121864319 s

In [13]:
cp = torch.load(os.path.join(root_dir, 'last.pt'))
model.load_state_dict(cp['state_dict'])
optimizer.load_state_dict(cp['optimizer'])
last_epoch_done = cp['epoch'] - 1

In [14]:
#continue training


if not os.path.exists(root_dir):
    os.makedirs(root_dir)


low_val_loss = 30000000
best_acc = -1
epoch = 0

val_loss_path = None
acc_path = None
#https://stackoverflow.com/questions/8078330/csv-writing-within-loop
import csv
with open(os.path.join(root_dir, 'logs2.csv'), 'w') as file_csv:
  writer=csv.writer(file_csv, delimiter=',',lineterminator='\n',)
  writer.writerow(['epoch', 'loss', 'acc', 'val_loss', 'val_acc'])


  for epoch in range(last_epoch_done, 121):
      row = [epoch]
      is_best = False
      running_loss = 0.0
      correct = 0
      total = 0

      model.train()

      t0 = time.time()
      for i, data in enumerate(training_loader):
          # get the inputs; data is a list of [inputs, labels]
          inputs, labels = data

          # for this_view in range(inputs.shape[1]):
          #   if np.random.randint(0, 4) == 0:
          #     inputs[:, this_view, ...] = 1.0

          inputs = inputs.to(device)
          labels = labels.to(device)

          # zero the parameter gradients
          optimizer.zero_grad()

          # forward + backward + optimize
          outputs = model(inputs)
          loss = criterion(outputs, labels)
          loss.backward()
          optimizer.step()

          # print statistics
          correct += (outputs.argmax(1) == labels).float().sum() #https://stackoverflow.com/questions/51503851/calculate-the-accuracy-every-epoch-in-pytorch
          total += labels.shape[0]
          running_loss += loss.item()

      print('epoch', epoch)
      
      print('{} seconds'.format(time.time() - t0))
      print('loss', running_loss)
      row.append(running_loss)
      running_loss = 0.0

      accuracy = 100 * correct / total
      print("Accuracy = {}".format(accuracy))
      row.append(torch.IntTensor.item(accuracy))
      correct = 0
      total= 0

      running_loss = 0.0
      correct = 0
      total = 0

      optimizer.zero_grad()
      model.eval()
      t0 = time.time()
      with torch.no_grad():
          for i, data in enumerate(val_loader):
              
              # get the inputs; data is a list of [inputs, labels]
              inputs, labels = data
              inputs = inputs.to(device)
              labels = labels.to(device)            


              # forward + backward + optimize
              outputs = model(inputs)
              loss = criterion(outputs, labels)

              correct += (outputs.argmax(1) == labels).float().sum() #https://stackoverflow.com/questions/51503851/calculate-the-accuracy-every-epoch-in-pytorch
              total += labels.shape[0]
              running_loss += loss.item()

      print('{} seconds'.format(time.time() - t0))
      print('val loss', running_loss)
      row.append(running_loss)
      

      accuracy = 100 * correct / total
      print(" Val Accuracy = {}".format(accuracy))
      row.append(torch.IntTensor.item(accuracy))

      model_name = 'ep_' + str(epoch) + '_loss_' + str(running_loss) + '_acc_' + str(accuracy) + '.pt'
      full_model_path = os.path.join(root_dir, model_name)
      checkpoint = {
          'epoch': epoch + 1,
          'state_dict': model.state_dict(),
          'optimizer': optimizer.state_dict()
      }
      if running_loss < low_val_loss:
        if val_loss_path and os.path.exists(val_loss_path):
          open(val_loss_path, 'w').close() #overwrite and make the file blank instead - ref: https://stackoverflow.com/a/4914288/3553367
          os.remove(val_loss_path)
        val_loss_path = full_model_path.replace('.pt', 'val_loss_best.pt')
        save_ckp(model, val_loss_path)
        low_val_loss = running_loss
        print('new low loss')

      if accuracy > best_acc:
        if acc_path and os.path.exists(acc_path):
          open(acc_path, 'w').close() #overwrite and make the file blank instead - ref: https://stackoverflow.com/a/4914288/3553367
          os.remove(acc_path)
        acc_path = full_model_path.replace('.pt', 'acc_best.pt')
        save_ckp(model, acc_path)
        best_acc = accuracy
        print('new best acc')
      full_model_path = os.path.join(root_dir, 'last.pt')
      save_ckp(checkpoint, full_model_path)
          

      

      correct = 0
      total= 0
      running_loss = 0.0
      writer.writerow(row)
  print('Finished Training')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:44: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


epoch 95
126.18246030807495 seconds
loss 771.2248685359955
Accuracy = 74.46394348144531
13.228357076644897 seconds
val loss 83.11411654949188
 Val Accuracy = 80.0
new low loss
new best acc
epoch 96
128.28181385993958 seconds
loss 771.6631615161896
Accuracy = 74.24116516113281
13.121023654937744 seconds
val loss 83.00200831890106
 Val Accuracy = 79.5
new low loss
epoch 97
127.68855357170105 seconds
loss 772.1304007768631
Accuracy = 73.87914276123047
13.21295690536499 seconds
val loss 83.53727698326111
 Val Accuracy = 79.0
epoch 98
126.02288246154785 seconds
loss 771.1183646917343
Accuracy = 74.38040161132812
13.221158742904663 seconds
val loss 84.19952189922333
 Val Accuracy = 77.75
epoch 99
126.67638945579529 seconds
loss 774.037003159523
Accuracy = 73.65636444091797
13.146863222122192 seconds
val loss 83.05145978927612
 Val Accuracy = 80.25
new best acc
epoch 100
128.73673129081726 seconds
loss 769.8658173084259
Accuracy = 74.74241638183594
13.417280197143555 seconds
val loss 82.95484